In [5]:
%load_ext autoreload
%autoreload 2
%xmode Verbose
import pandas as pd
from math import ceil, floor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Exception reporting mode: Verbose


In [6]:
from mtg_utils import (
    add_text_to_border_crop,
    get_border_crop,
    stack_images_vertically_with_overlap,
    concat_vertical,
)

In [7]:
def custom_round(number) -> int:
    return ceil(number) if number > 0 else floor(number)

In [8]:
alsa_data = pd.read_csv('data/alsa.csv')
card_data = pd.read_csv("data/card-ratings-2023-09-12-top.csv")
card_data['Color'] = card_data['Color'].fillna('C')
card_data = card_data.fillna("0%")
card_data['IWD'] = card_data['IWD'].str.replace(r'[^0-9-.]', '', regex=True).astype(float)
card_data['GIH WR'] = card_data['GIH WR'].str.replace(r'[^0-9-.]', '', regex=True).astype(float)
# apply custom_round to IWD
card_data['IWD'] = card_data['IWD'].apply(custom_round)
card_data['GIH WR'] = card_data['GIH WR'].apply(custom_round)
card_data['ATA'] = card_data['ATA'].apply(custom_round)
card_data['ALSA'] = card_data['ALSA'].apply(custom_round)

In [9]:
card_data = card_data[card_data['Rarity'].isin(['C', 'U'])]
#card_data = card_data[card_data['Rarity'].isin(['R', 'M'])]
card_data['rank'] = card_data['GIH WR'] + card_data['IWD']
card_data

,Name,Color,Rarity,# Seen,ALSA,# Picked,ATA,# GP,GP WR,# OH,OH WR,# GD,GD WR,# GIH,GIH WR,# GNS,GNS WR,IWD,rank
1,Archon's Glory,W,C,93010,8,14007,11,37948,56.7%,5902,56.6%,8695,57.8%,14597,58,23300,56.2%,2,60
2,Armory Mice,W,C,90514,8,13379,10,39252,56.0%,7081,55.7%,8189,53.0%,15270,55,23982,57.1%,-3,52
3,Besotted Knight,W,C,81726,6,11022,9,40821,54.2%,6837,54.5%,9464,54.7%,16301,55,24471,53.8%,1,56
4,Break the Spell,W,C,95299,8,10347,12,7717,50.4%,1233,48.7%,1738,51.4%,2971,51,4731,50.4%,-1,50
5,Charmed Clothier,W,C,85459,7,10327,10,30628,53.1%,4680,52.3%,7535,53.8%,12215,54,18404,53.0%,1,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,Raid Bombardment,R,U,35460,6,4550,9,9786,53.6%,1592,48.1%,1960,51.4%,3552,50,6220,55.8%,-6,44
313,Garruk's Uprising,G,U,34100,6,4648,8,11014,52.2%,1845,47.8%,2615,51.2%,4460,50,6544,53.9%,-5,45
314,Ground Seal,G,U,44499,8,5098,12,800,52.8%,144,0%,180,0%,324,0,476,0%,0,0
321,Season of Growth,G,U,36895,6,4362,9,7097,52.3%,1233,50.3%,1560,49.9%,2793,51,4302,53.7%,-4,47


In [10]:
groups = card_data.groupby('Color')
#groups = {c: g.sort_values(by=['GIH WR', 'ALSA'], ascending=[False, True]) for c, g in groups}
groups = {c: g.sort_values(by=['rank'], ascending=False) for c, g in groups}

In [11]:
MC = pd.concat([groups[mc] for mc, g in groups.items() if len(mc) > 1])
MC = MC.sort_values(by=["GIH WR", "ALSA"], ascending=[False, True])
W, U, B, R, G, C = [groups[c] for c in ["W", "U", "B", "R", "G", "C"]]
COLORS = [W, U, B, R, G, C, MC]

for df in COLORS:
    df["border_crop"] = df["Name"].iloc[:40].apply(get_border_crop)

for df in COLORS:
    df.iloc[:40].apply(
        lambda row: add_text_to_border_crop(
            row["border_crop"], row["ALSA"], row["GIH WR"], row["ATA"], row['IWD']
        ),
        axis=1,
    )

In [12]:
images = [stack_images_vertically_with_overlap(df['border_crop'].iloc[:40], 490) for df in COLORS]
new_img = concat_vertical(images)
new_img = new_img.resize((2560, int(2560*new_img.height/new_img.width)))
new_img.save('ranked_cu_2023-09-12-top.jpg')